# Fourier Transform

Author: Julian Lißner<br>
For questions and feedback write a mail to: [lissner@mib.uni-stuttgart.de](mailto:lissner@mib.uni-stuttgart.de)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from numpy.fft import fft, ifft
sys.path.extend( ['provided_functions', 'incomplete_functions', '../submodules'] )
import fourier_implementation as four
import result_check as check
from general_functions import tic, toc

## Discrete Fourier Transform (DFT)
- the DFT is defined as <br>
$$ X_n = \sum\limits_{k=0}^N x_k \exp{ (-2\jmath\pi k\, \frac nN)} $$
- $n$ loops of length $n$ $\blacktriangleright \mathcal O( n^2)$ 

---
__Task:__ Implement the `DFT_brute_force` in the 'fourier_implementation.py' file.

In [ ]:
x = np.arange( 16) #used to catch syntax errors
_ = four.DFT_brute_force( x)
check.FT_implementation( four.DFT_brute_force)

- the DFT's awful computational complexity can be improved
- $\exp{ (-2\jmath\pi k\,\frac nN)} $ are called twiddle factors
- twiddle factors can be preallocated as matrix <br>
$\quad \blacktriangleright$ computation of DFT via matrix multiplication
----
__Task:__ Implement the `DFT` in the 'fourier_implementation.py' file.

In [ ]:
_ = four.DFT( x)
check.FT_implementation( four.DFT)

## Fast Fourier Transform (FFT)

- FFT can be viewed as a recursive split and computation of the DFT
- the signal is recursively split up into odd and even parts<br>
- size $n$ is halfed on each split
- DFT $\blacktriangleright \mathcal O( n^2)\blacktriangleright$ call DFT on small $n$ (multiple splits)
- starting from lowest split level: recursively assemble even ($\cdot^e$) and odd ($\cdot^o$) parts <br>
$$ [ \hat x^e_k + \omega_n^k\, \hat x^o_k\qquad \,\hat x^e_k - \omega_n^k\, \hat x^o_k] $$
$$ \text{with}\quad \omega_n^k = \exp( -2\jmath\,\pi \,\frac kN ) \quad \text{and}\quad \hat x^{\ast} = \mathcal F( x^{\ast}) $$
$\quad\blacktriangleright$ compute DFT only on lowest level, recursively assemble left and right part

-------
__Task:__ Implement the Cooley-Tuckey algorithm of the FFT. <br>
OPTIONAL: if you are up for a challenge, implement the FFT without the function template.

In [ ]:
signal_length = 2**8
DFT_length = 2**4
signal = np.random.rand( signal_length) 

##OPTIONAL implementation without template
#check.FT_implementation( four.FFT_naked)

check.FT_implementation( four.FFT)

- the FFT performance is dependent on the size of the DFT and the signal length
- the numpy algorithm is significantly faster because other efficiency improvements have been implemented
- the tips and tricks video will hightlight a few of those

---
__Task:__ Compare the performance of the FFT dependent on signal length and DFT size.

In [ ]:
signal_length = [ 2**7, 2**8 ] #TODO adjust the length depending on your PC power
DFT_size = [ 2, 4] #TODO tweak the parameter
for length in signal_length:
    signal = np.random.rand( signal_length) 
    for DFT in DFT_size:
        tic_flag = 'performance check on signal length {} and DFT length {}'.format( legnth, DFT)
        tic( tic_flag, silent=True)
        result = #TODO #call your FFT function
        toc( tic_flag, precision=6 )
    tic( 'comparison to numpy fft, signal length:{}'.format( length), silent=True )
    result = #TODO #call the numpy FFT function
    toc( 'comparison to numpy fft, signal length:{}'.format( length), precision=6 ) 

------------
-----------
## Convolution Theorem
- the convolution theorem states
$$f \ast g = \mathcal F^{-1}\big( \, \mathcal F(f) \cdot \mathcal F(g) \,\big)$$

- that means algorithmically that a convolution of $\mathcal O( n^2)$ can be reduced to $\mathcal O( n\,\log n)$
- $f$ and $g$ have to be of same length for the implementation of the pointwise product
- keep in mind that the fourier transform induces periodicity
- _padding_ can circumvent periodic boundary effects $\blacktriangleright$ material of the next labs
- the convolution of two real valued signals is real valued. Imaginary parts are due to numerical precision and have to be removed, i.e. with `convolution.real`.
------
__Task:__ Apply a kernel with constant values $\frac1{\rm len_{\rm signal}}$ and the length of the signal via convolution to the signal.

In [ ]:
data =np.load( 'data/convolution_data.npz')
signal = data[ 'arr_0']
kernel = #TODO

flattened_signal = #TODO #convolution of signal with kernel

fig, axes = plt.subplots( 1, 2, figsize=(12, 8) )
axes[0].plot( signal, color='red', lw=2.5, label='original signal' )
axes[0].plot( kernel, color='black', lw=1.5, label='kernel' )
axes[1].plot( signal, color='red', ls='--', alpha=0.5, lw=1, label='original signal')
axes[1].plot( flattened_signal, color='red', lw=2.5, label='flattened signal')

for ax in axes:
    ax.legend()
    ax.grid( ls=':', color='#AAAAAA')
    ax.set_xlabel( 'x [-]')
    ax.set_ylabel( 'f(x) [-]')

- the convolution can be interpreted as:
    - how much a kernel is locally represented in the signal
    - placing a kernel on top of an excitiation given in the signal
    - neighbourhood averaging operations
- here the signal contains few entries (excitation) of different values (amplitues)
- at each excitation the full kernel (called stencil here) is placed
-------
__Task:__ Create the array with activation to apply the stencil on.

In [ ]:
stencil = data[ 'arr_1'] #Note that the shape of the stencil is about '16'  long
signal = np.zeros( stencil.shape)
print( 'length of the signal', len( signal))
activations = [ 2, 1 #TODO 
activation_location= [30, 47, #TODO
for i in range( len( activations)):
    signal[ activation_location[i] ] = activations[i]

activated_signal = #TODO #convolution of signal with stencil

fig, axes = plt.subplots( 1, 2, figsize=(12, 6) )
axes[0].plot( signal, color='red', lw=2, label='original signal' )
axes[0].plot( stencil, color='black', lw=1, label='kernel' )
axes[1].plot( stencil, color='red', ls='--', alpha=0.5, lw=2, label='original signal')
axes[1].plot( activated_signal, color='blue', lw=2, label='flattened signal')

for ax in axes:
    ax.legend()
    ax.grid( ls=':', color='#AAAAAA')
    ax.set_xlabel( 'x [-]')
    ax.set_ylabel( 'f(x) [-]')    


- a kernel in fourier space has to be centered at the 0th index
- the left half of the kernel should be placed at `kernel[-width//2:]` and the right half at `kernel[:width//2]`
- non centered kernels will introduce a translational shift in the result


----
__Task:__ Create yourself a triangle stencil of length 30, periodically centered at the 0th index. Apply the stencil to the signal. Match your convolution the loaded result.

In [ ]:
signal = data['arr_2']
result = data['arr_3']


stencil = #TODO
stencil#TODO #fill the triangle stencil
#TODO...
stencil =  #TODO #normalize the stencil s.t. sum( stencil) == 1

convoluted = #TODO #convolution of signal and stencil


fig, ax = plt.subplots( figsize=(7,6) )
ax.plot( convoluted, color='red', lw=2, label='computed convolution')
ax.plot( result, color='black', ls='--', lw=2, label='reference solution')

ax.legend()
ax.grid( ls=':', color='#AAAAAA')
ax.set_xlabel( 'x [-]')
ax.set_ylabel( 'f(x) [-]')    

assert np.allclose( result, convoluted), 'results do not match, please match the dashed line in the plot'